In [3]:
#Necessary for BeautifulSoup 
import sys
!{sys.executable} -m pip install geocoder

     |████████████████████████████████| 102kB 15.8MB/s ta 0:00:01
     |████████████████████████████████| 829kB 27.2MB/s eta 0:00:01
     |████████████████████████████████| 81kB 18.4MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [4]:
#Necessary for BeautifulSoup 
pip install BeautifulSoup4

     |████████████████████████████████| 112kB 20.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import libraries for use
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


In [5]:
# Data from internet using wikipedia URL
wikipedia_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page= requests.get(wikipedia_url)
 

# Use beautiful soup to parse codes
soup = BeautifulSoup(wikipedia_page.content, 'html5lib')
table = soup.find('div', attrs = {'id':'container'})


Extracting Table of WebPage Wikipedia

In [7]:
#Extracting raw table

table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# extracting a clean form of the table

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [8]:
#Integrating Postal codes with more than 1 neighbour
unique_p = set(Postcode)
print('quantity of unique Postal codes:', len(unique_p))
Postcode_uni      = []
Borough_uni       = []
Neighbourhood_uni = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_uni.append(p_var)
    Borough_uni.append(b_var)
    Neighbourhood_uni.append(n_var)

quantity of unique Postal codes: 78


In [11]:
#Create a Pandas Dataframe
toronto_dicti = {'Postcode':Postcode_uni, 'Borough':Borough_uni, 'Neighbourhood':Neighbourhood_uni}
df_toronto = pd.DataFrame.from_dict(toronto_dicti)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M9A,Etobicoke,Islington Avenue
1,M4H,East York,Thorncliffe Park
2,M1B,Scarborough,"Rouge, Malvern"
3,M9L,North York,Humber Summit
4,M4Y,Downtown Toronto,Church and Wellesley


In [13]:
#size dataframe
df_toronto.shape

(78, 3)